# Generate all possible homology classes corresponding to a bar
* This notebook contains illustrates how one can find all possible homology classes that correspond to a bar(s) under all possible interval decompositions of the persistence module 
* Code is from https://github.com/UDATG/analogous_bars
* data: samples from a square torus


In [1]:
using Revise
includet("../extension_method.jl")

WebIO._IJuliaInit()

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/PomtQ/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000107f6d2cf, Ptr{Nothing} @0x0000000108002206, Ptr{Nothing} @0x000000010800323b, Ptr{Nothing} @0x0000000108001b94, Ptr{Nothing} @0x0000000108001f0c, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010801c844, Ptr{Nothing} @0x000000010801bf65, Ptr{Nothing} @0x000000010801c604, Ptr{Nothing} @0x000000010801c604, Ptr{Nothing} @0x000000010801d134, Ptr{Nothing} @0x0000000110c024e7, Ptr{Nothing} @0x0000000107febfbf, Ptr{Nothing} @0x0000000110c53312, Ptr{Nothing} @0x0000000110889a65, Ptr{Nothing} @0x0000000107febfbf, Ptr{Nothing} @0x0000000107ff8839, Ptr{Nothing} @0x000000015315385e, Ptr{Nothing} @0x0000000107febfbf, Ptr{Nothing} @0x000000010800325f, Ptr{Nothing} @0x0000000108001b94, Ptr{Nothing} @0x000000

In [2]:
using .ext
using Distances
using Eirene
using JLD
using JLD2
using Plots
using Printf

Load example points and visualize
* `P`: points sampled from a trefoil knot in 3-dimensions
* `Q`: points sampled from a 2D projection of P

In [3]:
# load data
data = load("data/torus_two_samples/data.jld")

P_theta = data["P_theta"] # coordinates on square torus
P_phi = data["P_phi"];

Q = data["Q"] # coordinates on square torus
Q_theta = data["Q_theta"]
Q_phi = data["Q_phi"];


┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /Users/irisyoon/.julia/packages/FileIO/QkYgA/src/loadsave.jl:217


In [4]:
P_2d = hcat(P_theta, P_phi)
Q_2d = hcat(Q_theta, Q_phi)
p = plot_P_Q(P_2d, Q_2d)

Use distances computed on the square torus

In [5]:
# compute distance matrices
all_theta = vcat(P_theta, Q_theta)
all_phi = vcat(P_phi, Q_phi);
D = compute_distance_square_torus(all_theta, all_phi)
n = size(P_theta)[1]

# get submatrices
D_P = D[1:n, 1:n]
D_Q = D[n+1:end, n+1:end]
D_P_Q = D[1:n, n+1:end]
D_Q_P = D[n+1:end, 1:n];

Compute barcode of `P` in dimension 1

In [6]:
# run Eirene
C = eirene(D_P)

# plot barcode
dim = 1
barcode_P = barcode(C, dim = dim)
p = plot_barcode(barcode_P, lw = 3)
plot(p)

Select bar(s) of interest (hover over the above barcode)

In [7]:
selected_bars = [47]

# plot selected bar(s)
p = plot_barcode(barcode_P, selected_bars = selected_bars, lw = 5)
plot(p)

Find all homology classes that the select bar(s) can represent under different choices of basis of the persistence module.

In [8]:
all_classes = find_all_homology_classes(C, dim =1, selected_bars = selected_bars)

Dict{Any, Any} with 64 entries:
  [19, 20, 47]                => [101 16 … 31 69; 132 60 … 77 100]
  [4, 20, 21, 23, 47]         => [38 70 … 31 69; 70 125 … 77 100]
  [4, 21, 23, 47]             => [86 38 … 31 69; 125 86 … 77 100]
  [4, 19, 47]                 => [86 38 … 31 69; 125 86 … 77 100]
  [4, 22, 23, 47]             => [86 38 … 31 69; 125 86 … 77 100]
  [20, 23, 47]                => [85 47 … 31 69; 104 125 … 77 100]
  [19, 23, 47]                => [101 16 … 31 69; 132 60 … 77 100]
  [4, 21, 22, 47]             => [86 38 … 31 69; 125 86 … 77 100]
  [4, 20, 22, 23, 47]         => [38 70 … 31 69; 70 125 … 77 100]
  [47]                        => [38 13 … 31 69; 121 126 … 77 100]
  [20, 21, 22, 23, 47]        => [47 86 … 31 69; 125 125 … 77 100]
  [4, 20, 21, 22, 47]         => [38 70 … 31 69; 70 125 … 77 100]
  [4, 20, 21, 22, 23, 47]     => [38 70 … 31 69; 70 125 … 77 100]
  [19, 21, 22, 23, 47]        => [101 16 … 31 69; 132 60 … 77 100]
  [4, 19, 20, 23, 47]         => [38 7

* The keys of the dictionary are the possible representations of the selected bar in the same barcode with a different choice of basis. 
* The values of the dictionary represent the possible homology classes that the select bar(s) can represent under a different choice of basis.

Plot few of the possible homology classes

In [9]:
class = all_classes[[47]]
class = [class[:,i] for i in 1:size(class,2)]

plot_cycle_single_square_torus(P_2d, cycle = class, size = (300, 300))

In [10]:
class47 = all_classes[[47]]
class4_47 = all_classes[[4,47]]
class4_21_47 = all_classes[[4, 21, 47]]
class4_20_21_47 = all_classes[[4, 20, 21, 47]]

class47 = [class47[:,i] for i in 1:size(class47,2)]
class4_47 = [class4_47[:,i] for i in 1:size(class4_47,2)]
class4_21_47 = [class4_21_47[:,i] for i in 1:size(class4_21_47,2)]
class4_20_21_47 = [class4_20_21_47[:,i] for i in 1:size(class4_20_21_47,2)]

p2 = plot_cycle_single_square_torus(P_2d, cycle = class47, size = (300, 300), title = "class [47]")
p3 = plot_cycle_single_square_torus(P_2d, cycle = class4_47, size = (300, 300), title = "class [4,47]")
p4 = plot_cycle_single_square_torus(P_2d, cycle = class4_21_47, size = (300, 300), title = "class [4,21,47]")
p5 = plot_cycle_single_square_torus(P_2d, cycle = class4_20_21_47, size = (300, 300), title = "class [4,20,21,47]")

plot(p2, p3, p4, p5)